## Modules à importer

In [1]:
import math
import scipy
import pickle
import numpy as np
import pandas as pd
import iminuit as im
import ipywidgets as ipw
from variaIa import tools_ar
#from variaIa import tools_sc
from importlib import reload
from variaIa import stretchevol
from variaIa.tools_ar import surveys, df_cons, df_nc, df_c, df_full, z_max
#from variaIa.tools_sc import surveys, df_cons, df_nc, df_c, df_full, z_max
from IPython.display import display, Latex
import matplotlib.pyplot as plt
import random

## Définitions pandas, surveys et sélection de l'échantillon à utiliser

### Définition des données à montrer

In [2]:
def set_survey(cons, su):
    global df
    if su == 'All':
        df = df_cons(cons)
    elif 'n' in su:
        df = df_cons(cons)[df_cons(cons)['survey'] != su.split('n')[-1]]
    else:
        df = df_cons(cons)[df_cons(cons)['survey'] == su]
    return df

int_set_su = ipw.interactive(set_survey, cons=tools_ar.cons, su=tools_ar.su)
#int_set_su = ipw.interactive(set_survey, cons=tools_ar_sc.cons, su=tools_ar_sc.su)
display(int_set_su)

smap = plt.cm.get_cmap('cividis')
colors = {'SNF': plt.cm.binary(0.9),
          'SDSS': smap(0.1),
          'PS1': smap(0.5),
          'SNLS': smap(0.8),
          'HST': smap(0.99),  # for cividis
          #'HST': smap(300)}  # for viridis
          'All': (255/255, 51/255, 0/255)} 

def alphacolors(alpha, survey):
    return(colors[survey][0], colors[survey][1], colors[survey][2], alpha)

df_nHST = df_nc[df_nc['survey'] != 'HST']
surveysnl = surveys[0:1]+surveys[2:]

interactive(children=(Checkbox(value=False, description='Conservative'), Dropdown(description='Survey:', optio…

## Get proba

In [3]:
gen = stretchevol.generic()
gen.set_model('Evol3G2M2S')
base2STOT = gen.fit(df)

def get_proba(model):
    return np.exp((base2STOT.get_aic() - model.get_aic())/2)

## Modèles assymSURVEY

In [4]:
gen.set_model('Evol1G1M2S')

assymSNF = gen.fit(set_survey(tools_ar.cons.value, 'SNF'))

assymPS1 = gen.fit(set_survey(tools_ar.cons.value, 'PS1'))

assymSDSS = gen.fit(set_survey(tools_ar.cons.value, 'SDSS'))

assymSNLS = gen.fit(set_survey(tools_ar.cons.value, 'SNLS'))

assymHST = gen.fit(set_survey(tools_ar.cons.value, 'HST'))

assymTOT = gen.fit(set_survey(tools_ar.cons.value, 'nlow-z'))

assymTOT_nsnf = gen.fit(set_survey(tools_ar.cons.value, 'nSNF'))

## Modèles BaseSURVEY

In [5]:
gen.set_model('Evol3G2M2S')

baseSNF = gen.fit(assymSNF.pd)

baseSDSS = gen.fit(assymSDSS.pd)

basePS1 = gen.fit(assymPS1.pd)

baseSNLS = gen.fit(assymSNLS.pd)

baseHST = gen.fit(assymHST.pd)

baseTOT = gen.fit(assymTOT.pd)

baseTOT_nsnf = gen.fit(assymTOT_nsnf.pd)

In [25]:
modèles = [assymSNF, assymSDSS, assymPS1, assymSNLS, assymHST]

assym_comp = pd.DataFrame({'Assym': ['SNf', 'SDSS', 'PS1', 'SNLS', 'HST', 'Total'],
                           'Free param': [len(k.FREEPARAMETERS) for k in modèles] + [np.sum([len(k.FREEPARAMETERS) for k in modèles])],
                           '$\sigma^-$': [round(k.migrad_out[1][1][2],2) for k in modèles] + ['--'],
                           '$\sigma^-_{\mathrm{err}}$': [round(k.migrad_out[1][1][3],2) for k in modèles] + ['--'],
                           '$\sigma^+$': [round(k.migrad_out[1][2][2],2) for k in modèles] + ['--'],
                           '$\sigma^+_{\mathrm{err}}$': [round(k.migrad_out[1][2][3],2) for k in modèles] + ['--'],
                           '$\mu^0$': [round(k.migrad_out[1][0][2],2) for k in modèles] + ['--'],
                           '$\mu^0_{\mathrm{err}}$': [round(k.migrad_out[1][0][3],2) for k in modèles] + ['--'],
                           '$\mathcal{L}$': [round(k.get_logl(),1) for k in modèles] + [round(np.sum([k.get_logl() for k in modèles]),1)],
                           'AIC': [round(k.get_aic(),1) for k in modèles] + [round(np.sum([k.get_aic() for k in modèles]),1)]})

path = '../../../Data/tables/asym_comp/asym_comp-nHST'
if tools_ar.cons.value:
    path += '_cons'
#if tools_ar_sc.cons.value:
#    path += '_cons-supercut'
path += '.dat'
assym_comp.to_csv(path)

assym_comp

,Assym,Free param,$\sigma^-$,$\sigma^-_{\mathrm{err}}$,$\sigma^+$,$\sigma^+_{\mathrm{err}}$,$\mu^0$,$\mu^0_{\mathrm{err}}$,$\mathcal{L}$,AIC
0,SNf,3,1.34,0.13,-0.41,0.1,0.68,0.15,299.3,305.3
1,SDSS,3,1.7,0.28,-0,0.17,1.03,0.08,56.6,62.6
2,PS1,3,1.1,0.18,-0.55,0.17,0.32,0.25,178.7,184.7
3,SNLS,3,1.49,0.31,0,0.13,1.27,0.12,30.7,36.7
4,HST,3,0.76,0.36,0.79,0.35,0.11,0.44,73.7,79.7
5,Total,15,--,--,--,--,--,--,639.0,669.0


In [29]:
print('Delta AIC w/ BBC-modeling =',
      round(baseTOT.get_aic()-np.sum([k.get_aic() for k in modèles]),1))

Delta AIC w/ BBC-modeling = -43.9


In [42]:
print('Proba w/ BBC-modeling = ' +
      f'{np.exp((baseTOT.get_aic() - np.sum([k.get_aic() for k in modèles]))/2):.1e}')

Proba w/ BBC-modeling = 3.0e-10


## Another table comp

In [14]:
modelsa = [assymSDSS, assymPS1, assymSNLS, assymHST, assymSNF]
modelsb = [baseSDSS, basePS1, baseSNLS, baseHST, baseSNF]
param_sco = [[1.142, 1.652, 0.104], [0.384, 0.987, 0.505], [0.974, 1.236, 0.283]]

with open('../../Data/SNF_results', 'rb') as f:
    res_SNF = pickle.load(f)
    
with open('../../Data/ALL_results', 'rb') as f:
    res_ALL= pickle.load(f)

comp_L = pd.DataFrame({'#Data': [len(k.pd) for k in modelsa] +\
                       [np.sum([len(k.pd) for k in modelsa])] +\
                       [np.sum([len(k.pd) for k in modelsa[:-1]])],
                       'Survey': ['SDSS', 'PS1', 'SNLS', 'HST', 'SNf', 'Total', 'Total-SNf'],
                       '$\mathcal{L}$ asym (this)': [round(k.get_logl(),1) for k in modelsa] +\
                       [round(np.sum([k.get_logl() for k in modelsa]),1)] +\
                       [round(np.sum([k.get_logl() for k in modelsa[:-1]]),1)],
                       'Free param ': [len(k.FREEPARAMETERS) for k in modelsa] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-1]])],
                       '$\mathcal{L}$ Base All': [round(k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']),1) for k in modelsb] +\
                       [round(np.sum([k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']) for k in modelsb]),1)] +\
                       [round(np.sum([k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']) for k in modelsb[:-1]]),1)],
                       'Free param': [0, 0, 0, 0, 5,
                                      5,
                                      0],
                       '$\mathcal{L}$ asym Sco': [round(k.loglikelihood(p[0], p[1], p[2]),1) for k,p in zip(modelsa, param_sco)] + ['--'] + ['--']+\
                       ['--'] +\
                       [round(np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1)],
                      })

path = '../../Data/asym_comp_L'
if cons.value:
    path += '_cons-supercut'
path += '.dat'
comp_L.to_csv(path)

comp_L

,#Data,Survey,$\mathcal{L}$ asym (this),Free param,$\mathcal{L}$ Base All,Free param,$\mathcal{L}$ asym Sco
0,167,SDSS,445.0,3,446.6,0,457.9
1,160,PS1,397.4,3,399.4,0,397.6
2,102,SNLS,252.7,3,262.9,0,254.5
3,26,HST,73.7,3,75.0,0,--
4,114,SNf,299.3,3,272.9,5,--
5,569,Total,1468.2,15,1456.7,5,--
6,455,Total-SNf,1168.9,12,1183.8,0,1110


In [15]:
modelsa = [assymSDSS, assymPS1, assymSNLS, assymHST, assymSNF]
modelsb = [baseSDSS, basePS1, baseSNLS, baseHST, baseSNF]
param_sco = [[1.142, 1.652, 0.104], [0.384, 0.987, 0.505], [0.974, 1.236, 0.283]]
    
with open('../../Data/ALL_results', 'rb') as f:
    res_ALL = pickle.load(f)

comp_L = pd.DataFrame({'#Data': [len(k.pd) for k in modelsa] +\
                       [np.sum([len(k.pd) for k in modelsa])] +\
                       [np.sum([len(k.pd) for k in modelsa[:-1]])],
                       'Survey': ['SDSS', 'PS1', 'SNLS', 'HST', 'SNf', 'Total', 'Total-SNf'],
                       '$\mathcal{L}$ asym (this)': [round(k.get_logl(),1) for k in modelsa] +\
                       [round(np.sum([k.get_logl() for k in modelsa]),1)] +\
                       [round(np.sum([k.get_logl() for k in modelsa[:-1]]),1)],
                       'Free param ': [len(k.FREEPARAMETERS) for k in modelsa] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-1]])],
                       '$\mathcal{L}$ Base All': [round(k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']),1) for k in modelsb] +\
                       [round(np.sum([k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']) for k in modelsb]),1)] +\
                       [round(np.sum([k.loglikelihood(res_ALL['aa'], res_ALL['mu_1'], res_ALL['sigma_1'], res_ALL['mu_2'], res_ALL['sigma_2']) for k in modelsb[:-1]]),1)],
                       'Free param': [0, 0, 0, 0, 5,
                                      5,
                                      0],
                       '$\mathcal{L}$ asym Sco': [round(k.loglikelihood(p[0], p[1], p[2]),1) for k,p in zip(modelsa, param_sco)] + ['--'] + ['--']+\
                       ['--'] +\
                       [round(np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1)],
                      })

path = '../../Data/asym_comp_L_all'
if cons.value:
    path += '_cons'
path += '.dat'
comp_L.to_csv(path)

comp_L

,#Data,Survey,$\mathcal{L}$ asym (this),Free param,$\mathcal{L}$ Base All,Free param,$\mathcal{L}$ asym Sco
0,167,SDSS,445.0,3,446.6,0,457.9
1,160,PS1,397.4,3,399.4,0,397.6
2,102,SNLS,252.7,3,262.9,0,254.5
3,26,HST,73.7,3,75.0,0,--
4,114,SNf,299.3,3,272.9,5,--
5,569,Total,1468.2,15,1456.7,5,--
6,455,Total-SNf,1168.9,12,1183.8,0,1110


In [57]:
comp_A = pd.DataFrame({'#Data': [len(k.pd) for k in modelsa] +\
                       [np.sum([len(k.pd) for k in modelsa])] +\
                       [np.sum([len(k.pd) for k in modelsa[:-1]])] +\
                       [np.sum([len(k.pd) for k in modelsa[:-2]])],
                       'Survey': ['SDSS', 'PS1', 'SNLS', 'HST', 'SNf',
                                  'Total',
                                  'Total-SNf',
                                  'Total-SNf-HST'],
                       'AICc asym (this)': [round(k.get_aic(),1) for k in modelsa] +\
                       [round(np.sum([k.get_aic() for k in modelsa]),1)] +\
                       [round(np.sum([k.get_aic() for k in modelsa[:-1]]),1)] +\
                       [round(np.sum([k.get_aic() for k in modelsa[:-2]]),1)],
                       'Free param ': [len(k.FREEPARAMETERS) for k in modelsa] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-1]])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-2]])],
                       'AIC Base SNf': [round(k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']),1) for k in modelsb[:-1]] + [round(base.get_aic(),1)] +\
                       [round(2*5 + np.sum([k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']) for k in modelsb]),1)] +\
                       [round(np.sum([k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']) for k in modelsb[:-1]]),1)] +\
                       [round(np.sum([k.loglikelihood(res_SNF['aa'], res_SNF['mu_1'], res_SNF['sigma_1'], res_SNF['mu_2'], res_SNF['sigma_2']) for k in modelsb[:-2]]),1)],
                       'Free param': [0, 0, 0, 0, 5,
                                      5,
                                      0,
                                      0],
                       'AIC asym Sco': [round(2*3 + k.loglikelihood(p[0], p[1], p[2]),1) for k,p in zip(modelsa, param_sco)] + [80.8] + [305.5] +\
                       [round(2*15 + np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1) + 73.7 + 299.3] +\
                       [round(2*12 + np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1) + 73.7] +\
                       [round(2*9 + np.sum([k.loglikelihood(p[0], p[1], p[2]) for k,p in zip(modelsa, param_sco)]),1)],
                       ' Free param ': [len(k.FREEPARAMETERS) for k in modelsa] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-1]])] +\
                       [np.sum([len(k.FREEPARAMETERS) for k in modelsa[:-2]])],
                      })

path = '../Data/asym_comp_A'
if cons.value:
    path += '_cons'
path += '.dat'
comp_A.to_csv(path)

comp_A

,#Data,Survey,AICc asym (this),Free param,AIC Base SNf,Free param,AIC asym Sco,Free param
0,167,SDSS,451.0,3,452.9,0,463.9,3
1,160,PS1,403.4,3,405.2,0,403.6,3
2,102,SNLS,258.7,3,269.5,0,260.5,3
3,26,HST,79.7,3,75.9,0,80.8,3
4,114,SNf,305.3,3,278.6,5,305.5,3
5,569,Total,1498.2,15,1482.0,5,1513.0,15
6,455,Total-SNf,1192.9,12,1203.4,0,1207.7,12
7,429,Total-SNf-HST,1113.2,9,1127.5,0,1128.0,9


## Minimisation du modèle SNF et calcul $\mathcal{L}$ pour surveys

In [17]:
gen.set_model('Evol3G2M2S')
gen.set_data(df[df['survey'] == 'SDSS'])
baseSDSS = gen.fit()

gen.set_data(df[df['survey'] == 'PS1'])
basePS1 = gen.fit()

gen.set_data(df[df['survey'] == 'SNLS'])
baseSNLS = gen.fit()

In [18]:
models = [basePS1, baseSDSS, baseSNLS]

snf_comp = pd.DataFrame({'Base': ['SDSS', 'PS1', 'SNLS', 'Total'],
                         'Free param': [0 for k in models] + [0],
                         '$\mathcal{L}$': [round(k.get_logl(),1) for k in models] + [round(np.sum([k.get_logl() for k in models]),1)],
                         'AICc': [round(k.get_aic(),1) for k in models] + [round(np.sum([k.get_aic() for k in models]),1)]})

path = '../Data/snf_comp'
if cons.value:
    path += '_cons'
path += '.dat'
snf_comp.to_csv(path)

snf_comp

,Base,Free param,ln $\mathcal{L}$,AICc
0,SDSS,0,395.2,405.6
1,PS1,0,444.7,455.0
2,SNLS,0,254.1,264.7
3,Total,0,1094.0,1125.4
